<a href="https://colab.research.google.com/github/cometku/SEESMosquitoMappers/blob/master/csv_ceo_webscraper_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Webscraper Overview

Hello. First, thank you for running this program. This program will take your CEO Plots, take photos of the individual dots, and will fill a folder (created by the program) of these images categorically. This program will do most of the work: what you guys will need to do is direct the program to the right plots. This will be done through the program, in the form of prompts (the program will ask you to enter some information, and after you enter it, the program will send that to the site behind the scenes). The pieces of information you will need to enter are:

1. Your CEO account information (username and password)
2. The name of the institution (Globe Observer, in our case)
3. The project you want to scrape (a list will be printed, and you'll pick the project from the list)
4. The place in your drive that the folder (of the images) will be created
5. Whether you would like to see your analyzed plots (plot you've marked) or others' plots (which will be unmarked)
6. Which plot you would like to start at (enter the ID of the plot you've like to start at)
7. How many plots you'd like to scrape

Some of the cells in the notebook will take a while to finish running. This is okay. There might also be some visual glitches, but those don't affect anything either. If the cells **explicitly** throw an error (ValueError, TypeError, etc.), then something has gone wrong (and you can either email me, ananthmadan03@gmail.com, or DM me).

Just press Runtime>Run All (or Ctrl-F9) and let the program run. After all of the prompts have been answered (after you've entered how many plots you want to scrape), you can let the program run in the background while you work on something else. If you notice the program has gone idle (the CO icon should turn red), simple save and reload the page.

We hope that you will use this to scrape both your AOI and centerplot data from CEO.

Thank you.

P.S. the amount of images generated by this process amounts to approximately 32 MB of data.

In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 8.4MB/s 


In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 75.5 MB of archives.
After this operation, 256 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 83.0.4103.61-0ubuntu0.18.04.1 [1,119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 83.0.410

In [ ]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import re

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Disable GUIs for colab compatibility
options.add_argument('--no-sandbox') # Disable sandbox for third-party software to run
options.add_argument('--disable-dev-shm-usage') # Disable virtual memory storage

driver = webdriver.Chrome('chromedriver', options=options)
driver.get('https://collect.earth/login')
driver.set_window_size(1920,1200) #driver.maximize_window() was here before, it cut off part of the square and that's not very nice of them

# Login

In [ ]:
class ChangeToValidURL:
    
    def __init__(self, valid=[]):
        self.valid_urls = valid
    
    def __call__(self, driver):
        try:
            return driver.current_url in self.valid_urls
        except UnexpectedAlertPresentException:
            driver.switch_to.alert.accept() # Sometimes raises a NoAlertPresentException, I don't know why
            raise TimeoutException

Enter your email and password when prompted. This signs you in to your collect earth account so you can access your plots.

In [ ]:
from IPython.display import clear_output

# Generator for retry prompts
retry_prompts = iter([

    'you have two more chances',
    'one more chance',
    'Max amount of tries met.'
])

email_field = driver.find_element_by_xpath('//*[(@id = "email")]')
password_field = driver.find_element_by_xpath('//*[(@id = "password")]')

# Attempt login (3 times)
while True:

    # Prompt user for information
    email = input('Enter email: ')
    password = input('Enter password: ')

    email_field.send_keys(email) # Enter email
    password_field.send_keys(password) # Enter password

    clear_output(False) # Clear user information

    driver.find_element_by_xpath(
        '//*[contains(concat( " ", @class, " " ), concat( " ", "align-items-center", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "bg-lightgreen", " " ))]'
    ).click() # Click login button

    # If the page changes (login has succeeds)
    try:
        
        WebDriverWait(driver, 2).until(
            ChangeToValidURL(['https://collect.earth/home'])
        )

        print('Login succeeded')
        break
        
    # Otherwise, prompt user to retry login
    except (TimeoutException, NoAlertPresentException):

        retry_prompt = next(retry_prompts)

        # If the user has attempted login unsuccessfully 3 times
        if retry_prompt == 'Max amount of tries met.':

            print(retry_prompt)
            driver.close() # Close driver
            sys.exit(0)
        
        else:

            print('Incorrect login information.')
            print(f'Please, try again ({retry_prompt})')

            # Clear email and password fields
            email_field.send_keys(Keys.CONTROL + 'a')
            email_field.send_keys(Keys.DELETE)
            password_field.send_keys(Keys.CONTROL + 'a')
            password_field.send_keys(Keys.DELETE)

Login succeeded


# Machinated Human-like Search

Option to go through the site like a human would (clicking buttons, filling out forms, etc.) to find the desired project

This cell allows you to filter the institutions by entering what you type into the Collect Earth filter. Select an institution from what is filtered by entering the number.

In [ ]:
filter_elem = driver.find_element_by_xpath('//*[(@id = "filterInstitution")]')
while True:

    repeat = False
    filter_str = input('Enter institution name/filter (string): ')
    filter_elem.send_keys(filter_str)

    inst_dict = dict(enumerate(driver.find_elements_by_xpath("//p[contains(@class,'tree_label text-white')]"), 1))

    while True:

        clear_output(True)
        for i, inst in inst_dict.items():
            print('{0}. {1}'.format(i, inst.text.strip('\nⓘ'))) # Terrible to put this in the loop, but whatever

        choice = input('Select an institution (enter the number).\nIf you want to re-filter, enter nothing: ')

        if choice == '':

            filter_elem.send_keys(Keys.CONTROL + 'a')
            repeat = True
            break

        try:

            assert choice.isdigit() # User input must be a number
            choice = int(choice)
            assert choice >= 1 and choice <= len(inst_dict) # User input must be within range

            break

        except AssertionError:

            print(f'Invalid choice (must be a digit from 1 to {len(inst_dict)}')

    if not repeat:

        inst_dict[choice].find_element_by_xpath("//a[contains(@class,'institution_info btn')]").click() # Click the institution button
        break

1. GLOBE Observer
2. The GLOBE Program
Select an institution (enter the number).
If you want to re-filter, enter nothing: 1


Here, select the project from the project list by again entering the number corresponding to the project.

In [ ]:
# Get a list of all of the projects (found using xpath)
project_list = WebDriverWait(driver, 2).until(
    
    EC.presence_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "d-flex", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "text-truncate", " " ))]'))
)
project_dict = dict(enumerate(project_list, 1)) # Dictionary holding projects and a numerical ID

# Print project and ID
for i, project in project_dict.items():
    name = project.get_attribute('innerHTML') # Only contains project name
    print(f'{i}. {name}')

while True:
    
    # Prompt user for project select
    choice = input(f'Which project would you like to scrape (1 to {len(project_list)})? ')

    try:

        assert choice.isdigit() # User input must be a number
        choice = int(choice)
        assert choice >= 1 and choice <= len(project_list) # User input must be within range
    
    except AssertionError:

        print(f'Invalid choice (must be a digit from 1 to {len(project_list)}')
        continue
    
    # Click on the chosen project
    chosen = project_dict[choice]
    proj_name = chosen.get_attribute('innerHTML')
    chosen.click()
    break

1. GO LandChallenge part 1 (Sept13-Sept29)
2. GO LandChallenge part 1 BARREN OR NOT(Sept13-Sept29)
3. MHM-Peru-GMM-EarthDay
4. SEES2020-AOI (1-56)
5. SEES2020-AOI_centerPlot (1-56)
6. SEES2020-AOI (57-76)
7. SEES2020-AOI_centerPlot (57-76)
8. SEES2020-AOI (1-102)
9. SEES2020-AOI_centerPlot (1-102)
10. SEES2020-AOI (1-108)
11. SEES2020-AOI_centerPlot (1-108)
12. SEES2020-AOI_1to56_36GridOfInterest
13. SEES2020-AOI_57to108_36GridOfInterest
14. SEES2020-AOI_centerPlot (1-118)
15. SEES2020-AOI (1-118)
16. SEES2020-AOI_109to118_36GridOfInterest
17. SEES2020-AOI_centerPlot (1-118)_copy
18. SEES2020-AOI_109to118_36GridOfInterest_copy
19. SEES2020-LW_AM_AOI_1to56_36GridOfInterest
20. SEES2020-LW_AM_AOI_57to108_36GridOfInterest
21. SEES2020-LW_AM_AOI_109to118_36GridOfInterest
22. SEES2020-AOI (119-122)
23. SEES2020-AOI_centerPlot (119-122)
24. SEES2020-AOI_119to122_36GridOfInterest
25. SEES2020-AOI_5_36GridOfInterest
Which project would you like to scrape (1 to 25)? 19


# Plot Analysis



In [ ]:
import os

In [ ]:
while True:

    folder_path = input('Where would you like to save the plot images (must be a valid path name).\nIf you want to save at root, enter nothing: ')

    if folder_path != '':
        folder_path = f'/content/gdrive/{folder_path}/CEO_Plots/{proj_name}'
    else:
        folder_path = f'/content/gdrive/My Drive/CEO_Plots/{proj_name}'

    clear_output(True)

    try:

        if not os.path.isdir(folder_path):
            os.makedirs(folder_path)
            print(f'Made directory {folder_path}')
        else:
            print(f'Directory {folder_path} already exists.\nDefaulting to existing location.')

        break

    except OSError as e:
        print(f'Invalid path name: {e.args}')

Made directory /content/gdrive/My Drive/CEO_Plots/SEES2020-LW_AM_AOI_1to56_36GridOfInterest


When prompted, enter yes to scrape your analyzed plots.

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
files = files.upload()

Saving ceo-sees2020-aoi_1to56_36gridofinterest-sample-data-2020-07-25.csv to ceo-sees2020-aoi_1to56_36gridofinterest-sample-data-2020-07-25.csv


In [ ]:
prompt = 'Would you like to see your analyzed plots (yes or no)? '
valid_yes = 'yes'
valid_no = 'no'
while True:
    
    # Prompt user
    review = input(prompt).lower()
    
    if review != valid_yes and review != valid_no:

        print(f'Invalid answer (must be: {valid_yes} or {valid_no})')
        clear_output(True) # For readibility
        continue
    
    if review == '2':
        
        driver.close()
        sys.exit(0)

    # Accept different versions of 'yes'
    # The worst try-except block I've ever seen
    try:
        
        if review == valid_yes:
            
            if valid_yes == '1':
                
                driver.find_element_by_xpath(
        
                    '//*[contains(concat( " ", @class, " " ), concat( " ", "mt-2", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "btn-outline-darkgray", " " ))]'
                
                ).click()
            
            # Click on the 'review analyzed plots' form
            WebDriverWait(driver, 2).until(
            
                EC.element_to_be_clickable((By.CLASS_NAME, 'form-check'))
            
            ).click()

        WebDriverWait(driver, 5).until(
        
            EC.presence_of_element_located((By.ID, 'go-to-first-plot'))
        
        ).click()
        
        # Find the plot top select button (for plot identification)
        # if it appears within the first 5 seconds
        # Otherwise, a TimeoutError will be thrown
        # This still needs a handler
        top_slct_btn = WebDriverWait(driver, 20).until(
            
            EC.element_to_be_clickable((By.XPATH, '//*[(@id = "top-select")]'))
        
        )
        str_id = top_slct_btn.get_attribute('innerHTML') # Contains only the plot ID number
        
        break
    
    except TimeoutException as timeout:
        
        print(f'Element could not be located: {timeout}')
        continue
        
    except UnexpectedAlertPresentException as alert:
        
        print(re.search('(?<={).*(?=})', str(alert)).group()) # Alert message
        
        # New user prompt
        prompt = '''Would you like to:\n
                 1. Continue and instead view unanalyzed plots?\n
                 2. Exit program?\n'''
                 
        valid_yes = '1'
        valid_no = '2'

Would you like to see your analyzed plots (yes or no)? no


In [ ]:
# Re-click plot navigation button to re-open plot navigation submenu
driver.find_element_by_xpath(
    "(//h3[contains(@class,'ml-3 btn')])[1]"
).click()

In [ ]:
class ElementChange:

    def __init__(self, driver, locator, attr='innerHTML'):
        self.locator = locator
        self.attr = attr
        self.start_attr = EC._find_element(driver, locator).get_attribute(attr)

    def __call__(self, driver):

        try:
            return self.start_attr != EC._find_element(driver, self.locator).get_attribute(self.attr)
        except UnexpectedAlertPresentException:
            alert = driver.switch_to.alert
            print(alert.text)
            alert.accept()
            raise TimeoutException

Start at your first plot number. If the number given and your first plot match, press enter to start. Otherwise, you would need to enter your first plot.

In [ ]:
while True:

    id_elem = driver.find_element_by_id('plotId')
    plt_id = re.search('\d+', driver.find_element_by_xpath("(//h3[@class='ml-2'])[1]").get_attribute('innerHTML')).group()

    plt_num = input(f'Enter the plot ID of the plot you\'d like to go to (integer).\n \
                    If you want to start at the current plot, enter nothing.\n \
                    (Current plot ID is {plt_id}): ')
    
    if plt_num == '':
        break

    try:
        id_elem.send_keys((Keys.CONTROL + 'a'))
        id_elem.send_keys(plt_num)
        driver.find_element_by_xpath("(//button[@class='btn btn-outline-lightgreen'])[2]").click()

        WebDriverWait(driver, 10).until(
            ElementChange(driver, (By.XPATH, "(//h3[@class='ml-2'])[1]"))
        )

        plt_id = re.search('\d+', driver.find_element_by_xpath("(//h3[@class='ml-2'])[1]").get_attribute('innerHTML')).group()

        print(f'Current plot is now: {plt_id}')

        break
    
    except (TimeoutException, NoAlertPresentException):

        clear_output(True)

Enter the plot ID of the plot you'd like to go to (integer).
                     If you want to start at the current plot, enter nothing.
                     (Current plot ID is 101): 


In [ ]:
lc_elems = driver.find_elements_by_xpath(

    '//*[contains(concat( " ", @class, " " ), concat( " ", "pl-1", " " ))]'

)

color_to_lc = dict(zip(
    
    list(map(
        
        lambda x: tuple(map(int, x.split(', '))),
        [
            re.search('(?<=rgb\().*(?=\);)', lc_elem.get_attribute('innerHTML')).group()
            for lc_elem in lc_elems
        ]
    )),
    
    list(map(
        
        lambda x: x.replace('&gt;', '>').replace('/', '_or_').title(),
        [
            re.search('(?<=<span class="small">).*(?=</span)', lc_elem.get_attribute('innerHTML')).group()
            for lc_elem in lc_elems
        ]
    ))
))


In [ ]:
for land_cover in color_to_lc.values():

    try:
        os.mkdir(f'{folder_path}/{land_cover}')
        print(f'Created subdirectory {land_cover}')
    except OSError:
        print(f'Subdirectory {land_cover} already exists.\nDefaulting to existing subdirectory.')

Created subdirectory Trees_Canopycover
Created subdirectory Bush_Or_Scrub
Created subdirectory Grass
Created subdirectory Cultivated Vegetation
Created subdirectory Water>Treated Pool
Created subdirectory Water>Lake_Or_Ponded_Or_Container
Created subdirectory Water>Rivers_Or_Stream
Created subdirectory Water>Irrigation Ditch
Created subdirectory Shadow
Created subdirectory Unknown
Created subdirectory Bare Ground
Created subdirectory Building
Created subdirectory Impervious Surface (No Building)


In [ ]:
import time

In [ ]:
zoom_in = driver.find_element_by_xpath(
    '//*[contains(concat( " ", @class, " " ), concat( " ", "ol-zoom-in", " " ))]'
)
for _ in range(4):
    zoom_in.click()
    time.sleep(0.1)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
def bytes_to_image(bytes_):
    array = np.frombuffer(bytes_, dtype=np.uint8)
    return cv2.imdecode(array, cv2.IMREAD_ANYCOLOR)

In [ ]:
def get_plot_image(driver, pos, size, pad=(200, 50)):
    # This might need padding added to the plot size to shrink to overall plot size down
    # so the chance that cv classifies a random circle that isn't a circle goes down
    return bytes_to_image(driver.get_screenshot_as_png())[pos[1]+pad[1]:pos[1]+size[1]-pad[1], pos[0]+pad[0]:pos[0]+size[0]-pad[0]]

In [ ]:
def move_sequence(driver, elem, dx_dy):
# I got it working. It now can click and drag on the screen
    actions = ActionChains(driver)

    actions.move_to_element_with_offset(elem, half_size[0]-10, half_size[1]-10)
    actions.click_and_hold()
    actions.move_by_offset(10, 10) # This doesn't do anything movement-wise; I assume it just updates the GUI
                                   # Whatever it does, it works. Thank you Larry

    actions.move_by_offset(*dx_dy) # We fancy out here with these positional args
    actions.release()
    actions.perform()

In [ ]:
# Move the canvas distances that would normally be out of bounds
# by sub-dividing the distance into chunks that lie within bounds
# And then moving by the total offset at the end
def move_canvas(driver, elem, half_size, dx_dy, pad=15):

    signs = np.sign(dx_dy)
    dx_dy = np.abs(dx_dy)

    size = tuple(map(lambda x: x-pad, half_size))

    mod_dict = dict([(d//s, [d, s]) for s, d in zip(size, dx_dy)])

    i = max(mod_dict)
    d, s = mod_dict[i]

    if i != 0:

        if d == dx_dy[0]:

            step = np.multiply(signs, (s, dx_dy[1]//i))
            offset = np.multiply(signs, (d%s, dx_dy[1]-i*dx_dy[1]//i))

        else:

            step = np.multiply(signs, (dx_dy[0]//i, s))
            offset = np.multiply(signs, (dx_dy[0]-i*dx_dy[0]//i, d%s))

    else:

        step = (0, 0)
        offset = np.multiply(signs, dx_dy)

    for _ in range(i):
        move_sequence(driver, elem, step)
    move_sequence(driver, elem, offset)
    ActionChains(driver).move_by_offset(-offset[0], -offset[1])

In [ ]:
def get_unloaded_sum(im, unloaded_color=(52, 115, 117)):
    return (im == unloaded_color).all(axis=-1).astype(int).sum()

In [ ]:
elem = driver.find_element_by_xpath('//canvas')
pos = tuple(elem.location.values())
size = tuple(reversed(tuple(elem.size.values())))
half_size = tuple(map(lambda x: x//2, size))

This cell finds the distance between each dot. The expect output should be somewhere in between 1060 and 1080. This cell might take some time; just let it run.

In [ ]:
pt_dist, pt_loc = get_pt_dist(driver, elem, pos, size, half_size, bgr_set)
print(pt_dist)

NameError: ignored

This cell finds the width of the plot (in dots) using the distance between each point (found in the previous cell). This one might also take some time.

In [ ]:
plt_width = get_plot_width(driver, elem, pos, size, half_size, pt_dist, bgr_set)
print(plt_width)

NameError: ignored

In [ ]:
num_dots = plt_width**2
print(num_dots)

NameError: ignored

In [ ]:
move_canvas(driver, elem, half_size, (0, pt_dist*(plt_width//2))) # Move to the top-right corner

NameError: ignored

Enter the amount of plots you would like to scrape. Please scrape as many as you can, 36 for most.

In [ ]:
while True:

    num_plots = input('How many plots do you want to scrape.\nIf you want to scrape as many as possible, enter nothing: ')
    clear_output(True)
    
    if num_plots == '':
        num_plots = np.inf # I have mixed feelings about this
        break

    try:
        assert num_plots.isdigit()
        num_plots = int(num_plots)
        assert num_plots > 0
        break
    
    except AssertionError:
        print('Invalid input: input must be an integer greater than 0')

In [ ]:
plt_id = re.search('"\d+"', id_elem.get_attribute('outerHTML')).group().strip('"')
start_id = plt_id
finished = 0
while True:
    # Snake traversal
    for i in range(num_dots):
        
        print(f'\rCurrently processing dot #{i+1}', end='', flush=True)
        # Wait until image is mostly loaded
        for j in range(8):
            plt_im = get_plot_image(driver, pos, size)
            
            if j == 0:
                print(f'\rStalling until dot {i+1} has loaded fully', end='', flush=True)

            if get_unloaded_sum(plt_im) <= 10 or j > 8:
                print(f'\rDot {i+1} has finished loading', end='', flush=True)
                break

        im, cat = crop_and_label(plt_im, pt_loc, bgr_set)
        if cat is None:
            print(f'\rPlot {plt_id} is unanalyzed')
            break

        im = cv2.resize(im, (64, 64), interpolation=cv2.INTER_AREA) # Down-scaling image

        im_name = f'{plt_id}_{i}'
        cv2.imwrite(f'{folder_path}/{cat}/{im_name}.png', im)

        if (i+1)%11 == 0:
            step = (pt_dist, 0)
        else:
            step = (0, pt_dist*(-1)**(i//11 + 1))

        move_canvas(driver, elem, half_size, step)

        print(f'\rFinished processing dot #{i+1}', end='', flush=True)

    print(f'\rFinished analyzing plot {plt_id}')
    finished += 1

    if plt_id == start_id or finished >= num_plots:
            print('Exiting')
            print(f'Plot Count: {finished}')
            print(f'Start Plot: {start_id}')
            print(f'End Plot: {plt_id}')
            break

    clear_output(True)
    
    for _ in range(2):

        try:
            
            driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "mx-1", " " ))]').click() # Click the button to go to the next plot

            WebDriverWait(driver, 10).until(
                ElementChange(driver, (By.XPATH, "(//h3[@class='ml-2'])[1]"))
            )
            break
        
        except (TimeoutException, NoAlertPresentException):

            while True:

                end = False
                x = input('Would you like to continue (yes or no)? ').lower()

                if x == 'yes':

                    print('Switching analysis mode')
                    WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, 'form-check'))
                    ).click()
                    start_id = None # Reset the starting id
                    break

                elif x == 'no':

                    print('Ending')
                    end = True # Very ham-fisted, but works (I think)
                    break

                else:

                    print('Invalid response: must be "yes" or "no"')

            if end:
                break

    id_elem = driver.find_element_by_id('plotId')
    plt_id = re.search('"\d+"', id_elem.get_attribute('outerHTML')).group().strip('"')
    print(f'Now processing plot {plt_id}')

    if start_id is None:
        start_id = plt_id

    # Zoom in to dot level
    for _ in range(4):
        zoom_in.click()
        time.sleep(0.1)

    move_canvas(driver, elem, half_size, (-pt_dist*(plt_width//2), pt_dist*(plt_width//2)), 50) # Get to the top-right corner

If the program disconnects, reload the page. The program should be still working. If it does not appear to be so (the above cell has no dynamic output), then simply restart the program and resume at the current plot (if the program stopped at plot 4528, simply resume, via the initial plot selection cell, at 4528.)

In [ ]:
driver.close()

After you are finished, share the folder of the plots with me (ananthmadan03@gmail.com).

# Everything

In [ ]:
# -*- coding: utf-8 -*-
"""csv_ceo_webscraper_colab.ipynb
 
Automatically generated by Colaboratory.
 
Original file is located at
    https://colab.research.google.com/drive/1FopeJCOTjx9trqiCKvKHcWzlublIpLLc
 
# Webscraper Overview
 
Hello. First, thank you for running this program. This program will take your CEO Plots, take photos of the individual dots, and will fill a folder (created by the program) of these images categorically. This program will do most of the work: what you guys will need to do is direct the program to the right plots. This will be done through the program, in the form of prompts (the program will ask you to enter some information, and after you enter it, the program will send that to the site behind the scenes). The pieces of information you will need to enter are:
 
1. Your CEO account information (username and password)
2. The name of the institution (Globe Observer, in our case)
3. The project you want to scrape (a list will be printed, and you'll pick the project from the list)
4. The place in your drive that the folder (of the images) will be created
5. Whether you would like to see your analyzed plots (plot you've marked) or others' plots (which will be unmarked)
6. Which plot you would like to start at (enter the ID of the plot you've like to start at)
7. How many plots you'd like to scrape
 
Some of the cells in the notebook will take a while to finish running. This is okay. There might also be some visual glitches, but those don't affect anything either. If the cells **explicitly** throw an error (ValueError, TypeError, etc.), then something has gone wrong (and you can either email me, ananthmadan03@gmail.com, or DM me).
 
Just press Runtime>Run All (or Ctrl-F9) and let the program run. After all of the prompts have been answered (after you've entered how many plots you want to scrape), you can let the program run in the background while you work on something else. If you notice the program has gone idle (the CO icon should turn red), simple save and reload the page.
 
We hope that you will use this to scrape both your AOI and centerplot data from CEO.
 
Thank you.
 
P.S. the amount of images generated by this process amounts to approximately 32 MB of data.
"""
 
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
 
import re
 
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Disable GUIs for colab compatibility
options.add_argument('--no-sandbox') # Disable sandbox for third-party software to run
options.add_argument('--disable-dev-shm-usage') # Disable virtual memory storage
 
driver = webdriver.Chrome('chromedriver', options=options)
driver.get('https://collect.earth/login')
driver.set_window_size(1920,1200) #driver.maximize_window() was here before, it cut off part of the square and that's not very nice of them
 
"""# Login"""
 
class ChangeToValidURL:
    
    def __init__(self, valid=[]):
        self.valid_urls = valid
    
    def __call__(self, driver):
        try:
            return driver.current_url in self.valid_urls
        except UnexpectedAlertPresentException:
            driver.switch_to.alert.accept() # Sometimes raises a NoAlertPresentException, I don't know why
            raise TimeoutException
 
"""Enter your email and password when prompted. This signs you in to your collect earth account so you can access your plots."""
 
from IPython.display import clear_output
 
# Generator for retry prompts
retry_prompts = iter([
 
    'you have two more chances',
    'one more chance',
    'Max amount of tries met.'
])
 
email_field = driver.find_element_by_xpath('//*[(@id = "email")]')
password_field = driver.find_element_by_xpath('//*[(@id = "password")]')
 
# Attempt login (3 times)
while True:
 
    # Prompt user for information
    email = input('Enter email: ')
    password = input('Enter password: ')
 
    email_field.send_keys(email) # Enter email
    password_field.send_keys(password) # Enter password
 
    clear_output(False) # Clear user information
 
    driver.find_element_by_xpath(
        '//*[contains(concat( " ", @class, " " ), concat( " ", "align-items-center", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "bg-lightgreen", " " ))]'
    ).click() # Click login button
 
    # If the page changes (login has succeeds)
    try:
        
        WebDriverWait(driver, 2).until(
            ChangeToValidURL(['https://collect.earth/home'])
        )
 
        print('Login succeeded')
        break
        
    # Otherwise, prompt user to retry login
    except (TimeoutException, NoAlertPresentException):
 
        retry_prompt = next(retry_prompts)
 
        # If the user has attempted login unsuccessfully 3 times
        if retry_prompt == 'Max amount of tries met.':
 
            print(retry_prompt)
            driver.close() # Close driver
            sys.exit(0)
        
        else:
 
            print('Incorrect login information.')
            print(f'Please, try again ({retry_prompt})')
 
            # Clear email and password fields
            email_field.send_keys(Keys.CONTROL + 'a')
            email_field.send_keys(Keys.DELETE)
            password_field.send_keys(Keys.CONTROL + 'a')
            password_field.send_keys(Keys.DELETE)
 
"""# Machinated Human-like Search
 
Option to go through the site like a human would (clicking buttons, filling out forms, etc.) to find the desired project
 
This cell allows you to filter the institutions by entering what you type into the Collect Earth filter. Select an institution from what is filtered by entering the number.
"""
 
filter_elem = driver.find_element_by_xpath('//*[(@id = "filterInstitution")]')
while True:
 
    repeat = False
    filter_str = input('Enter institution name/filter (string): ')
    filter_elem.send_keys(filter_str)
 
    inst_dict = dict(enumerate(driver.find_elements_by_xpath("//p[contains(@class,'tree_label text-white')]"), 1))
 
    while True:
 
        clear_output(True)
        for i, inst in inst_dict.items():
            print('{0}. {1}'.format(i, inst.text.strip('\nⓘ'))) # Terrible to put this in the loop, but whatever
 
        choice = input('Select an institution (enter the number).\nIf you want to re-filter, enter nothing: ')
 
        if choice == '':
 
            filter_elem.send_keys(Keys.CONTROL + 'a')
            repeat = True
            break
 
        try:
 
            assert choice.isdigit() # User input must be a number
            choice = int(choice)
            assert choice >= 1 and choice <= len(inst_dict) # User input must be within range
 
            break
 
        except AssertionError:
 
            print(f'Invalid choice (must be a digit from 1 to {len(inst_dict)}')
 
    if not repeat:
 
        inst_dict[choice].find_element_by_xpath("//a[contains(@class,'institution_info btn')]").click() # Click the institution button
        break
 
"""Here, select the project from the project list by again entering the number corresponding to the project."""
 
# Get a list of all of the projects (found using xpath)
project_list = WebDriverWait(driver, 2).until(
    
    EC.presence_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "d-flex", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "text-truncate", " " ))]'))
)
project_dict = dict(enumerate(project_list, 1)) # Dictionary holding projects and a numerical ID
 
# Print project and ID
for i, project in project_dict.items():
    name = project.get_attribute('innerHTML') # Only contains project name
    print(f'{i}. {name}')
 
while True:
    
    # Prompt user for project select
    choice = input(f'Which project would you like to scrape (1 to {len(project_list)})? ')
 
    try:
 
        assert choice.isdigit() # User input must be a number
        choice = int(choice)
        assert choice >= 1 and choice <= len(project_list) # User input must be within range
    
    except AssertionError:
 
        print(f'Invalid choice (must be a digit from 1 to {len(project_list)}')
        continue
    
    # Click on the chosen project
    chosen = project_dict[choice]
    proj_name = chosen.get_attribute('innerHTML')
    chosen.click()
    break
 
"""# Plot Analysis"""
 
import os
 
while True:
 
    folder_path = input('Where would you like to save the plot images (must be a valid path name).\nIf you want to save at root, enter nothing: ')
 
    if folder_path != '':
        folder_path = f'/content/gdrive/{folder_path}/CEO_Plots/{proj_name}'
    else:
        folder_path = f'/content/gdrive/My Drive/CEO_Plots/{proj_name}'
 
    clear_output(True)
 
    try:
 
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path)
            print(f'Made directory {folder_path}')
        else:
            print(f'Directory {folder_path} already exists.\nDefaulting to existing location.')
 
        break
 
    except OSError as e:
        print(f'Invalid path name: {e.args}')
 
"""When prompted, enter yes to scrape your analyzed plots."""
 
import pandas as pd
 
lc_csv = pd.read_csv(r'C://Users/user/Documents/Python Scripts/sees/ceo-sees2020-aoi_57to108_36gridofinterest-sample-data-2020-07-25.csv')
lc_csv = lc_csv.dropna(subset=['LAND COVER ELEMENTS'])
 
lc_dict = {}
for i, row in lc_csv.iterrows():
    lc_dict.setdefault(row['PL_PLOTID'], []).append(row['LAND COVER ELEMENTS'])
 
prompt = 'Would you like to see your analyzed plots (yes or no)? '
valid_yes = 'yes'
valid_no = 'no'
while True:
    
    # Prompt user
    review = input(prompt).lower()
    
    if review != valid_yes and review != valid_no:
 
        print(f'Invalid answer (must be: {valid_yes} or {valid_no})')
        clear_output(True) # For readibility
        continue
    
    if review == '2':
        
        driver.close()
        sys.exit(0)
 
    # Accept different versions of 'yes'
    # The worst try-except block I've ever seen
    try:
        
        if review == valid_yes:
            
            if valid_yes == '1':
                
                driver.find_element_by_xpath(
        
                    '//*[contains(concat( " ", @class, " " ), concat( " ", "mt-2", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "btn-outline-darkgray", " " ))]'
                
                ).click()
            
            # Click on the 'review analyzed plots' form
            WebDriverWait(driver, 2).until(
            
                EC.element_to_be_clickable((By.CLASS_NAME, 'form-check'))
            
            ).click()
 
        WebDriverWait(driver, 5).until(
        
            EC.presence_of_element_located((By.ID, 'go-to-first-plot'))
        
        ).click()
        
        # Find the plot top select button (for plot identification)
        # if it appears within the first 5 seconds
        # Otherwise, a TimeoutError will be thrown
        # This still needs a handler
        top_slct_btn = WebDriverWait(driver, 20).until(
            
            EC.element_to_be_clickable((By.XPATH, '//*[(@id = "top-select")]'))
        
        )
        str_id = top_slct_btn.get_attribute('innerHTML') # Contains only the plot ID number
        
        break
    
    except TimeoutException as timeout:
        
        print(f'Element could not be located: {timeout}')
        continue
        
    except UnexpectedAlertPresentException as alert:
        
        print(re.search('(?<={).*(?=})', str(alert)).group()) # Alert message
        
        # New user prompt
        prompt = '''Would you like to:\n
                 1. Continue and instead view unanalyzed plots?\n
                 2. Exit program?\n'''
                 
        valid_yes = '1'
        valid_no = '2'
 
# Re-click plot navigation button to re-open plot navigation submenu
driver.find_element_by_xpath(
    "(//h3[contains(@class,'ml-3 btn')])[1]"
).click()
 
class ElementChange:
 
    def __init__(self, driver, locator, attr='innerHTML'):
        self.locator = locator
        self.attr = attr
        self.start_attr = EC._find_element(driver, locator).get_attribute(attr)
 
    def __call__(self, driver):
 
        try:
            return self.start_attr != EC._find_element(driver, self.locator).get_attribute(self.attr)
        except UnexpectedAlertPresentException:
            alert = driver.switch_to.alert
            print(alert.text)
            alert.accept()
            raise TimeoutException
 
"""Start at your first plot number. If the number given and your first plot match, press enter to start. Otherwise, you would need to enter your first plot."""
 
while True:
 
    id_elem = driver.find_element_by_id('plotId')
    plt_id = re.search('\d+', driver.find_element_by_xpath("(//h3[@class='ml-2'])[1]").get_attribute('innerHTML')).group()
 
    plt_num = input(f'Enter the plot ID of the plot you\'d like to go to (integer).\n \
                    If you want to start at the current plot, enter nothing.\n \
                    (Current plot ID is {plt_id}): ')
    
    if plt_num == '':
        break
 
    try:
        id_elem.send_keys((Keys.CONTROL + 'a'))
        id_elem.send_keys(plt_num)
        driver.find_element_by_xpath("(//button[@class='btn btn-outline-lightgreen'])[2]").click()
 
        WebDriverWait(driver, 10).until(
            ElementChange(driver, (By.XPATH, "(//h3[@class='ml-2'])[1]"))
        )
 
        plt_id = re.search('\d+', driver.find_element_by_xpath("(//h3[@class='ml-2'])[1]").get_attribute('innerHTML')).group()
 
        print(f'Current plot is now: {plt_id}')
 
        break
    
    except (TimeoutException, NoAlertPresentException):
 
        clear_output(True)
 
lc_elems = driver.find_elements_by_xpath(
 
    '//*[contains(concat( " ", @class, " " ), concat( " ", "pl-1", " " ))]'
 
)
 
color_to_lc = dict(zip(
    
    list(map(
        
        lambda x: tuple(map(int, x.split(', '))),
        [
            re.search('(?<=rgb\().*(?=\);)', lc_elem.get_attribute('innerHTML')).group()
            for lc_elem in lc_elems
        ]
    )),
    
    list(map(
        
        lambda x: x.replace('&gt;', '>').replace('/', '_or_').title(),
        [
            re.search('(?<=<span class="small">).*(?=</span)', lc_elem.get_attribute('innerHTML')).group()
            for lc_elem in lc_elems
        ]
    ))
))
 
for land_cover in color_to_lc.values():
 
    try:
        os.mkdir(f'{folder_path}/{land_cover}')
        print(f'Created subdirectory {land_cover}')
    except OSError:
        print(f'Subdirectory {land_cover} already exists.\nDefaulting to existing subdirectory.')
 
import time
 
zoom_in = driver.find_element_by_xpath(
    '//*[contains(concat( " ", @class, " " ), concat( " ", "ol-zoom-in", " " ))]'
)
for _ in range(4):
    zoom_in.click()
    time.sleep(0.1)
 
import numpy as np
import cv2
 
def bytes_to_image(bytes_):
    array = np.frombuffer(bytes_, dtype=np.uint8)
    return cv2.imdecode(array, cv2.IMREAD_ANYCOLOR)
 
def get_plot_image(driver, pos, size, pad=(200, 50)):
    # This might need padding added to the plot size to shrink to overall plot size down
    # so the chance that cv classifies a random circle that isn't a circle goes down
    return bytes_to_image(driver.get_screenshot_as_png())[pos[1]+pad[1]:pos[1]+size[1]-pad[1], pos[0]+pad[0]:pos[0]+size[0]-pad[0]]
 
def move_sequence(driver, elem, dx_dy):
# I got it working. It now can click and drag on the screen
    actions = ActionChains(driver)
 
    actions.move_to_element_with_offset(elem, half_size[0]-10, half_size[1]-10)
    actions.click_and_hold()
    actions.move_by_offset(10, 10) # This doesn't do anything movement-wise; I assume it just updates the GUI
                                   # Whatever it does, it works. Thank you Larry
 
    actions.move_by_offset(*dx_dy) # We fancy out here with these positional args
    actions.release()
    actions.perform()
 
# Move the canvas distances that would normally be out of bounds
# by sub-dividing the distance into chunks that lie within bounds
# And then moving by the total offset at the end
def move_canvas(driver, elem, half_size, dx_dy, pad=15):
 
    signs = np.sign(dx_dy)
    dx_dy = np.abs(dx_dy)
 
    size = tuple(map(lambda x: x-pad, half_size))
 
    mod_dict = dict([(d//s, [d, s]) for s, d in zip(size, dx_dy)])
 
    i = max(mod_dict)
    d, s = mod_dict[i]
 
    if i != 0:
 
        if d == dx_dy[0]:
 
            step = np.multiply(signs, (s, dx_dy[1]//i))
            offset = np.multiply(signs, (d%s, dx_dy[1]-i*dx_dy[1]//i))
 
        else:
 
            step = np.multiply(signs, (dx_dy[0]//i, s))
            offset = np.multiply(signs, (dx_dy[0]-i*dx_dy[0]//i, d%s))
 
    else:
 
        step = (0, 0)
        offset = np.multiply(signs, dx_dy)
 
    for _ in range(i):
        move_sequence(driver, elem, step)
    move_sequence(driver, elem, offset)
    ActionChains(driver).move_by_offset(-offset[0], -offset[1])
 
def get_unloaded_sum(im, unloaded_color=(52, 115, 117)):
    return (im == unloaded_color).all(axis=-1).astype(int).sum()
 
elem = driver.find_element_by_xpath('//canvas')
pos = tuple(elem.location.values())
size = tuple(reversed(tuple(elem.size.values())))
half_size = tuple(map(lambda x: x//2, size))
 
"""This cell finds the distance between each dot. The expect output should be somewhere in between 1060 and 1080. This cell might take some time; just let it run."""
 
pt_dist, pt_loc = 1072, None
 
"""This cell finds the width of the plot (in dots) using the distance between each point (found in the previous cell). This one might also take some time."""
 
plt_width = 11
 
num_dots = plt_width**2
print(num_dots)
 
move_canvas(driver, elem, half_size, (0, pt_dist*(plt_width//2))) # Move to the top-right corner
 
"""Enter the amount of plots you would like to scrape. Please scrape as many as you can, 36 for most."""
 
while True:
 
    num_plots = input('How many plots do you want to scrape.\nIf you want to scrape as many as possible, enter nothing: ')
    clear_output(True)
    
    if num_plots == '':
        num_plots = np.inf # I have mixed feelings about this
        break
 
    try:
        assert num_plots.isdigit()
        num_plots = int(num_plots)
        assert num_plots > 0
        break
    
    except AssertionError:
        print('Invalid input: input must be an integer greater than 0')
 
plt_id = re.search('"\d+"', id_elem.get_attribute('outerHTML')).group().strip('"')
start_id = plt_id
finished = 0
 
'''This needs so much checking it\'s unreal. Check everything on a small scale, then scale it up.'''
lc_ids = iter(lc_dict)
while True:
    # Snake traversal
    dots = 1
    for i in range(plt_width):
        
        if not i%2:
            r = range(plt_width)
        else:
            r = range(plt_width-1, -1, -1)
        for j in r:
        
            print(f'\rCurrently processing dot #{dots}', end='', flush=True)
            # Wait until image is mostly loaded
            for k in range(8):
                plt_im = get_plot_image(driver, pos, size)
                
                if j == 0:
                    print(f'\rStalling until dot #{dots} has loaded fully', end='', flush=True)
    
                if get_unloaded_sum(plt_im) <= 10 or k > 8:
                    print(f'\rDot #{dots} has finished loading', end='', flush=True)
                    break
    
            im = get_plot_image(driver, pos, size)
    
            im = cv2.resize(im, (64, 64), interpolation=cv2.INTER_AREA) # Down-scaling image
            cat = lc_dict[plt_id][j+i*11]
    
            im_name = f'{plt_id}_{j+i*11}'
            cv2.imwrite(f'{folder_path}/{cat}/{im_name}.png', im)
            
            # ========== Check steps so it goes from BL->TR ==========
            if not (i+1)%11:
                step = (pt_dist*(-1)**(i//11 + 1), 0)
            else:
                step = (0, pt_dist)
            # ========================================================
    
            move_canvas(driver, elem, half_size, step)
    
            print(f'\rFinished processing dot #{i+1}', end='', flush=True)
 
    print(f'\rFinished analyzing plot {plt_id}')
    finished += 1
 
    if plt_id == start_id or finished >= num_plots:
            print('Exiting')
            print(f'Plot Count: {finished}')
            print(f'Start Plot: {start_id}')
            print(f'End Plot: {plt_id}')
            break
 
    clear_output(True)
    
    # Need something here that gets the next key (id)
    # from lc_dict and enters it into the plot navigator
    # Entering the id can be done using the lc_iter
    plt_id = next(lc_ids)
    id_elem = driver.find_element_by_id('plotId')
    id_elem.send_keys((Keys.CONTROL + 'a'))
    id_elem.send_keys(plt_id)
    driver.find_element_by_xpath("(//button[@class='btn btn-outline-lightgreen'])[2]").click()
 
    for _ in range(2):
 
        try:
            
            driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "mx-1", " " ))]').click() # Click the button to go to the next plot
 
            WebDriverWait(driver, 10).until(
                ElementChange(driver, (By.XPATH, "(//h3[@class='ml-2'])[1]"))
            )
            break
        
        except (TimeoutException, NoAlertPresentException):
 
            while True:
 
                end = False
                x = input('Would you like to continue (yes or no)? ').lower()
 
                if x == 'yes':
 
                    print('Switching analysis mode')
                    WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, 'form-check'))
                    ).click()
                    start_id = None # Reset the starting id
                    break
 
                elif x == 'no':
 
                    print('Ending')
                    end = True # Very ham-fisted, but works (I think)
                    break
 
                else:
 
                    print('Invalid response: must be "yes" or "no"')
 
            if end:
                break
            
    plt_id = re.search('"\d+"', id_elem.get_attribute('outerHTML')).group().strip('"')
    print(f'Now processing plot {plt_id}')
 
    if start_id is None:
        start_id = plt_id
 
    # Zoom in to dot level
    for _ in range(4):
        zoom_in.click()
        time.sleep(0.1)
 
    move_canvas(driver, elem, half_size, (-pt_dist*(plt_width//2), pt_dist*(plt_width//2)), 50) # Get to the top-right corner
 
"""If the program disconnects, reload the page. The program should be still working. If it does not appear to be so (the above cell has no dynamic output), then simply restart the program and resume at the current plot (if the program stopped at plot 4528, simply resume, via the initial plot selection cell, at 4528.)"""
 
driver.close()
 
"""After you are finished, share the folder of the plots with me (ananthmadan03@gmail.com)."""